In [2]:
!pip install -q libauc

In [1]:
import json
import time
from pathlib import Path
from io import BytesIO

import albumentations as A
import h5py
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from accelerate import Accelerator
from accelerate.utils import (
    DistributedDataParallelKwargs,
    ProjectConfiguration,
    set_seed,
)
from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.metrics import auc, roc_curve, roc_auc_score
from timm import create_model
from torch.utils.data import DataLoader, Dataset
from libauc.sampler import DualSampler
from libauc.losses import pAUCLoss
from libauc.optimizers import SOPAs

from isic_helper import DotDict, get_folds

In [2]:
def dev_augment(image_size):
    transform = A.Compose(
        [
            A.Transpose(p=0.5),
            A.VerticalFlip(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(
                brightness_limit=0.2, contrast_limit=0.2, p=0.75
            ),
            A.OneOf(
                [
                    A.MotionBlur(blur_limit=(5, 7)),
                    A.MedianBlur(blur_limit=(5, 7)),
                    A.GaussianBlur(blur_limit=(5, 7)),
                    A.GaussNoise(var_limit=(5.0, 30.0)),
                ],
                p=0.7,
            ),
            A.OneOf(
                [
                    A.OpticalDistortion(distort_limit=1.0),
                    A.GridDistortion(num_steps=5, distort_limit=1.0),
                    A.ElasticTransform(alpha=3),
                ],
                p=0.7,
            ),
            A.CLAHE(clip_limit=4.0, p=0.7),
            A.HueSaturationValue(
                hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5
            ),
            A.ShiftScaleRotate(
                shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85
            ),
            A.Resize(image_size, image_size),
            A.CoarseDropout(
                max_height=int(image_size * 0.375),
                max_width=int(image_size * 0.375),
                max_holes=1,
                min_holes=1,
                p=0.7,
            ),
            A.Normalize(),
            ToTensorV2(),
        ],
        p=1.0,
    )
    return transform


def val_augment(image_size):
    transform = A.Compose(
        [A.Resize(image_size, image_size), A.Normalize(), ToTensorV2()], p=1.0
    )
    return transform


class ISICDataset(Dataset):
    def __init__(self, metadata, images, augment, infer=False):
        self.metadata = metadata
        self.images = images
        self.targets = metadata["target"].values
        self.augment = augment
        self.length = len(self.metadata)
        self.infer = infer
        if not infer:
            self.value_counts_dict = self.metadata.target.value_counts().to_dict()
            self.imratio = self.value_counts_dict[1]/self.value_counts_dict[0]

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        row = self.metadata.iloc[index]

        image = np.array(Image.open(BytesIO(self.images[row["isic_id"]][()])))
        image = self.augment(image=image)["image"]

        data = image.float()

        if not self.infer:
            target = torch.tensor(row["target"]).long()
            return data, target, index

        return data, index


class ISICNet(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super(ISICNet, self).__init__()
        self.model = create_model(
            model_name=model_name,
            pretrained=pretrained,
            in_chans=3,
            num_classes=0,
            global_pool="",
        )
        self.classifier = nn.Linear(self.model.num_features, 1)
        self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])

    def forward(self, image):
        x = self.model(image)
        bs = len(image)
        pool = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)

        if self.training:
            logit = 0
            for i in range(len(self.dropouts)):
                logit += self.classifier(self.dropouts[i](pool))
            logit = logit / len(self.dropouts)
        else:
            logit = self.classifier(pool)
        return logit

    
def train_epoch(
    epoch,
    model,
    optimizer,
    criterion,
    dev_dataloader,
    accelerator,
    log_interval=10,
):
    model.train()
    train_loss = []
    total_steps = len(dev_dataloader)
    for step, (data, target, index) in enumerate(dev_dataloader):
        optimizer.zero_grad()
        logits = model(data)
        probs = torch.sigmoid(logits)
        loss = criterion(probs, target, index)
        accelerator.backward(loss)
        optimizer.step()

        loss_value = accelerator.gather(loss).item()
        train_loss.append(loss_value)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        if step % log_interval == 0:
            print(
                f"Epoch: {epoch} | Step: {step + 1}/{total_steps} |"
                f" Loss: {loss_value:.5f} | Smooth loss: {smooth_loss:.5f}"
            )
    train_loss = np.mean(train_loss)
    return train_loss


def get_trans(img, iteration):
    if iteration >= 6:
        img = img.transpose(2, 3)
    if iteration % 6 == 0:
        return img
    elif iteration % 6 == 1:
        return torch.flip(img, dims=[2])
    elif iteration % 6 == 2:
        return torch.flip(img, dims=[3])
    elif iteration % 6 == 3:
        return torch.rot90(img, 1, dims=[2, 3])
    elif iteration % 6 == 4:
        return torch.rot90(img, 2, dims=[2, 3])
    elif iteration % 6 == 5:
        return torch.rot90(img, 3, dims=[2, 3])


def val_epoch(
    epoch,
    model,
    criterion,
    val_dataloader,
    accelerator,
    n_tta,
    log_interval=50,
):
    model.eval()
    val_probs = []
    val_targets = []
    val_loss = []
    total_steps = len(val_dataloader)
    with torch.no_grad():
        for step, (data, target, index) in enumerate(val_dataloader):
            logits = 0
            probs = 0
            for idx in range(n_tta):
                logits_iter = model(get_trans(data, idx))
                logits += logits_iter
                probs += torch.sigmoid(logits_iter)
            logits /= n_tta
            probs /= n_tta

            loss = criterion(probs, target, index)
            val_loss.append(loss.detach().cpu().numpy())

            logits, probs, targets = accelerator.gather((logits, probs, target))
            val_probs.append(probs)
            val_targets.append(targets)

            if step % log_interval == 0:
                print(f"Epoch: {epoch} | Step: {step + 1}/{total_steps}")

    val_loss = np.mean(val_loss)
    val_probs = torch.cat(val_probs).cpu().numpy()
    val_targets = torch.cat(val_targets).cpu().numpy()
    val_auc = compute_auc(val_targets, val_probs)
    val_pauc = compute_pauc(val_targets, val_probs, min_tpr=0.8)
    return (
        val_loss,
        val_auc,
        val_pauc,
        val_probs,
        val_targets
    )

def compute_auc(y_true, y_pred) -> float:
    """
    Compute the Area Under the Receiver Operating Characteristic Curve (ROC AUC).

    Args:
        y_true: ground truth of 1s and 0s
        y_pred: predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, 1]
    """
    return roc_auc_score(y_true, y_pred)


def compute_pauc(y_true, y_pred, min_tpr: float = 0.80) -> float:
    """
    2024 ISIC Challenge metric: pAUC

    Given a solution file and submission file, this function returns the
    partial area under the receiver operating characteristic (pAUC)
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        min_tpr:
        y_true: ground truth of 1s and 0s
        y_pred: predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    """

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(y_true - 1)

    # flip the submissions to their compliments
    v_pred = -1.0 * y_pred

    max_fpr = abs(1 - min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

    #     # Equivalent code that uses sklearn's roc_auc_score
    #     v_gt = abs(np.asarray(solution.values)-1)
    #     v_pred = np.array([1.0 - x for x in submission.values])
    #     max_fpr = abs(1-min_tpr)
    #     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    #     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    #     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    #     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    return partial_auc


In [3]:
INPUT_DIR = Path("/kaggle/input")
ARTIFACTS_DIR = Path(".")

args = DotDict()

args.data_dir = INPUT_DIR / "isic-2024-challenge"
args.model_name = "efficientnet_b1"
args.version = "v1"
args.model_identifier = f"deep_auc_{args.model_name}_{args.version}"
args.model_dir = ARTIFACTS_DIR / args.model_identifier
args.model_dir.mkdir(parents=True, exist_ok=True)
args.pretrained_model_path = INPUT_DIR / f"isic-scd-{args.model_name.replace('_', '-')}-{args.version}-train"
args.logging_dir = "logs"
args.fold = 1

args.mixed_precision = "fp16"
args.image_size = 128
args.batch_size = 64
args.num_workers = 4
args.init_lr = 1e-3
args.gamma = 500
args.margin = 1.0
args.weight_decay = 1e-5
args.num_epochs = 10
args.n_tta: int = 6
args.seed = 2022
args.debug = True

In [4]:
logging_dir = Path(args.model_dir, args.logging_dir)
accelerator_project_config = ProjectConfiguration(
    project_dir=args.model_dir, logging_dir=str(logging_dir)
)
kwargs = DistributedDataParallelKwargs()
accelerator = Accelerator(
    mixed_precision=args.mixed_precision,
    project_config=accelerator_project_config,
    kwargs_handlers=[kwargs],
)
print(accelerator.state)

if args.seed is not None:
    set_seed(args.seed)

Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16



In [5]:
train_metadata = pd.read_csv(args.data_dir / "train-metadata.csv", low_memory=False)
test_metadata = pd.read_csv(args.data_dir / "test-metadata.csv")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_images = h5py.File(args.data_dir / "train-image.hdf5", mode="r")
test_images = h5py.File(args.data_dir / "test-image.hdf5", mode="r")

Train data size: (401059, 57)
Test data size: (3, 44)


In [6]:
if args.debug:
    train_metadata = train_metadata.sample(frac=0.05, random_state=args.seed)

In [7]:
dev_index = train_metadata[train_metadata["fold"] != args.fold].index
val_index = train_metadata[train_metadata["fold"] == args.fold].index

dev_metadata = train_metadata.loc[dev_index, :].reset_index(drop=True)
val_metadata = train_metadata.loc[val_index, :].reset_index(drop=True)

dev_dataset = ISICDataset(
    dev_metadata, train_images, augment=dev_augment(args.image_size)
)
val_dataset = ISICDataset(
    val_metadata, train_images, augment=val_augment(args.image_size)
)

sampling_rate = 0.1
sampler = DualSampler(dev_dataset, args.batch_size, sampling_rate=sampling_rate)

dev_dataloader = DataLoader(
    dev_dataset,
    batch_size=args.batch_size,
    sampler=sampler,
    num_workers=args.num_workers,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
    drop_last=False,
    pin_memory=True,
)

model = ISICNet(
    model_name=args.model_name, pretrained=False,
)
model = model.to(accelerator.device)

criterion = pAUCLoss("1w", data_len=len(dev_dataset), margin=args.margin, gamma=args.gamma)
optimizer = SOPAs(model.parameters(), 
                 mode='adam', 
                 lr=args.init_lr, 
                 weight_decay=args.weight_decay)

(
    model,
    optimizer,
    dev_dataloader,
    val_dataloader,
) = accelerator.prepare(
    model, optimizer, dev_dataloader, val_dataloader
)

model_filepath = args.pretrained_model_path / f"models/fold_{args.fold}"
accelerator.load_state(model_filepath)

best_val_auc = 0
best_val_pauc = 0
best_val_loss = 0
best_epoch = 0
best_val_probs = None

for epoch in range(1, args.num_epochs + 1):
    print(f"Fold {args.fold} | Epoch {epoch}")
    start_time = time.time()

    train_loss = train_epoch(
        epoch,
        model,
        optimizer,
        criterion,
        dev_dataloader,
        accelerator,
    )
    (
        val_loss,
        val_auc,
        val_pauc,
        val_probs,
        val_targets,
    ) = val_epoch(
        epoch,
        model,
        criterion,
        val_dataloader,
        accelerator,
        args.n_tta,
    )

    if val_pauc > best_val_pauc:
        print(
            f"pAUC: {best_val_pauc:.5f} --> {val_pauc:.5f}, saving model..."
        )
        best_val_pauc = val_pauc
        best_val_auc = val_auc
        best_val_loss = val_loss
        best_epoch = epoch
        best_val_probs = binary_probs
        output_dir = f"{args.model_dir}/models/fold_{args.fold}"
        accelerator.save_state(output_dir)
    else:
        print(
            f"pAUC: {best_val_pauc:.5f} --> {val_pauc:.5f}, skipping model save..."
        )
    print(
        f"Fold: {args.fold} | Epoch: {epoch} |"
        f" Train loss: {train_loss:.5f} | Val loss: {val_loss:.5f}"
        f" Val AUC: {val_auc:.5f} | Val pAUC: {val_pauc:.5f}"
    )
    elapsed_time = time.time() - start_time
    elapsed_mins = int(elapsed_time // 60)
    elapsed_secs = int(elapsed_time % 60)
    print(f"Epoch {epoch} took {elapsed_mins}m {elapsed_secs}s")

print(
    f"Fold: {args.fold} | "
    f"Best Val pAUC: {best_val_pauc} | Best AUC: {best_val_auc} |"
    f" Best loss: {best_val_loss} |"
    f" Best epoch: {best_epoch}"
)
oof_df = pd.DataFrame(
    {
        "isic_id": val_metadata["isic_id"],
        "patient_id": val_metadata["patient_id"],
        "fold": args.fold,
        "target": val_metadata["target"],
        f"oof_{args.model_identifier}": best_val_probs,
    }
)
oof_df.to_csv(
    f"{args.model_dir}/oof_preds_{args.model_identifier}_fold_{args.fold}.csv",
    index=False,
)

fold_metadata = {
    "fold": args.fold,
    "best_epoch": best_epoch,
    "best_val_auc": best_val_auc,
    "best_val_pauc": best_val_pauc,
    "best_val_loss": float(best_val_loss),
}
with open(f"{args.model_dir}/models/fold_{args.fold}/metadata.json", "w") as f:
    json.dump(fold_metadata, f)
print(f"Finished training fold {args.fold}")


RuntimeError: Error(s) in loading state_dict for ISICNet:
	size mismatch for classifier.weight: copying a param with shape torch.Size([9, 1280]) from checkpoint, the shape in current model is torch.Size([1, 1280]).
	size mismatch for classifier.bias: copying a param with shape torch.Size([9]) from checkpoint, the shape in current model is torch.Size([1]).